In [ ]:
#import nltk
#nltk.download('stopwords')

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
import plotly.graph_objs as go
#import plotly.plotly as py
#import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
#cufflinks.go_offline()
#cufflinks.set_config_file(world_readable=True, theme='pearl')

Using TensorFlow backend.


In [2]:
df1 = pd.read_csv('data_sgd/train.tsv', header=None, sep="\t")

In [3]:
df1.head()

,0,1
0,__label__1,"sounds good, i want to buy tickets."
1,__label__3,please search a hotel for me there
2,__label__4,nothing. just relax.
3,__label__5,shall i book a table here?
4,__label__6,"no, not now.i need to check my account balance?"


In [4]:
df2 = pd.read_csv('data_sgd/test.tsv', header=None, sep="\t")
df2.head()

,0,1
0,__label__23,sounds good. can you find me a rental car ther...
1,__label__26,i need to check weather on the 5th of march.
2,__label__24,cool. schedule a visit.
3,__label__14,i would like to get a cab going there.
4,__label__1,would you like some tickets?


In [5]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 25000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 80#250
# This is fixed.
EMBEDDING_DIM = 100
HIDDEN_DIM = 64

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df1[1].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 4476 unique tokens.


In [6]:
X1 = tokenizer.texts_to_sequences(df1[1].values)
X1 = pad_sequences(X1, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X1.shape)

Shape of data tensor: (47022, 80)


In [7]:
Y1 = pd.get_dummies(df1[0]).values
print('Shape of label tensor:', Y1.shape)

Shape of label tensor: (47022, 21)


In [8]:
X2 = tokenizer.texts_to_sequences(df2[1].values)
X2 = pad_sequences(X2, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X2.shape)

Shape of data tensor: (9838, 80)


In [9]:
Y2 = pd.get_dummies(df2[0]).values
print('Shape of label tensor:', Y2.shape)

Shape of label tensor: (9838, 21)


In [10]:
X_train, X_test, Y_train, Y_test = X1,X2, Y1,Y2
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(47022, 80) (47022, 21)
(9838, 80) (9838, 21)


In [11]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [12]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X1.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(SimpleRNN(HIDDEN_DIM, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(Y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc',f1_m,precision_m, recall_m]) # metrics=['accuracy']
print(model.summary())





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 80, 100)           2500000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 80, 100)           0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 64)                10560     
_________________________________________________________________
dense_1 (Dense)              (None, 21)                1365      
Total params: 2,511,925
Trainable params: 2,511,925
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 42319 samples, validate on 4703 samples
Epoch 1/5





42319/42319 [==============================] - 55s 1ms/step - loss: 1.7567 - acc: 0.5118 - f1_m: 0.4112 - precision_m: 0.8292 - recall_m: 0.2944 - val_loss: 0.8482 - val_acc: 0.7763 - val_f1_m: 0.7317 - val_precision_m: 0.8947 - val_recall_m: 0.6209
Epoch 2/5
42319/42319 [==============================] - 54s 1ms/step - loss: 0.7635 - acc: 0.7819 - f1_m: 0.7608 - precision_m: 0.8839 - recall_m: 0.6710 - val_loss: 0.5646 - val_acc: 0.8437 - val_f1_m: 0.8372 - val_precision_m: 0.8882 - val_recall_m: 0.7925
Epoch 3/5
42319/42319 [==============================] - 54s 1ms/step - loss: 0.5657 - acc: 0.8314 - f1_m: 0.8268 - precision_m: 0.8858 - recall_m: 0.7761 - val_loss: 0.5110 - val_acc: 0.8560 - val_f1_m: 0.8583 - val_precision_m: 0.8933 - val_recall_m: 0.8265
Epoch 4/5
42319/42319 [==============================] - 54s 1ms/step

In [14]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

9838/9838 [==============================] - 2s 225us/step
Test set
  Loss: 0.440
  Accuracy: 0.877


In [15]:
#loss, accuracy, f1_score, precision, recall
print(accr)

[0.43979792595273187, 0.8773124618461448, 0.8813795566219556, 0.9004542142376954, 0.8636918072415504]


In [16]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
print(classification_report(np.argmax(Y_test, axis=1), y_pred_bool, digits=4))

9838/9838 [==============================] - 2s 245us/step
              precision    recall  f1-score   support

           0     0.7971    0.8015    0.7993       549
           1     0.8512    0.9687    0.9062       319
           2     0.8763    0.9444    0.9091       180
           3     0.8969    0.9667    0.9305       270
           4     0.8356    0.9592    0.8931       196
           5     0.8394    0.8120    0.8255       367
           6     0.8355    0.4618    0.5948       275
           7     0.8757    0.7381    0.8010       210
           8     0.8974    0.9277    0.9123       415
           9     0.8742    0.8797    0.8770       158
          10     0.6871    0.9218    0.7873       243
          11     0.8828    0.9496    0.9150       357
          12     0.7919    0.7548    0.7729       363
          13     0.8003    0.9376    0.8635       577
          14     0.8000    0.7023    0.7480       393
          15     0.9713    0.9531    0.9621       213
          16     0.965